In [124]:
%matplotlib ipympl
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pytransform3d.rotations import *

In [125]:
# create two views of points
pts1 = np.array([[0.5, 0.5],
                 [0.6, 0.6],
                 [0.7, 0.7],
                 [0.65, 0.75]])
pts2 = np.array([[0.4, 0.5],
                 [0.5, 0.6],
                 [0.6, 0.4],
                 [0.6, 0.65]])

In [137]:
pts1.T

array([[0.5 , 0.6 , 0.7 , 0.65],
       [0.5 , 0.6 , 0.7 , 0.75]])

In [127]:
# camera matrix
K = np.array([[1,0,0.5],[0,1,0.5],[0,0,1]])

In [128]:
# relative camera pose
R1 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]).astype(np.float64)
t1 = np.array([[0], [0], [0]]).astype(np.float64)
R2 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]).astype(np.float64)
t2 = np.array([[1], [0], [0]]).astype(np.float64)

In [129]:
# create projection matrices
proj_matrix1 = np.hstack([R1.T, -R1.T.dot(t1)])
proj_matrix2 = np.hstack([R2.T, -R2.T.dot(t2)])
proj_matrix1 = K.dot(proj_matrix1)
proj_matrix2 = K.dot(proj_matrix2)

In [130]:
proj_matrix1

array([[1. , 0. , 0.5, 0. ],
       [0. , 1. , 0.5, 0. ],
       [0. , 0. , 1. , 0. ]])

In [131]:
proj_matrix2

array([[ 1. ,  0. ,  0.5, -1. ],
       [ 0. ,  1. ,  0.5,  0. ],
       [ 0. ,  0. ,  1. ,  0. ]])

In [132]:
pts = cv2.triangulatePoints(proj_matrix1, proj_matrix2, pts1.T, pts2.T).T

In [133]:
pts = cv2.convertPointsFromHomogeneous(pts).reshape(-1, 3)

In [134]:
pts

array([[-4.94065646e-323, -0.00000000e+000,  1.00000000e+001],
       [ 1.00000000e+000,  1.00000000e+000,  1.00000000e+001],
       [ 1.79588498e+000,  4.28337839e-001,  8.38445426e+000],
       [ 2.94885129e+000,  3.91628023e+000,  1.95354482e+001]])

In [141]:
# pt1
#img_points = np.array([[0.5, 0.5],
#                 [0.6, 0.6],
#                 [0.7, 0.7],
#                 [0.65, 0.75]])
# pt2
img_points = np.array([[0.4, 0.5],
                 [0.5, 0.6],
                 [0.6, 0.4],
                 [0.6, 0.65]])

retval, rvec, tvec, inliers = cv2.solvePnPRansac(pts.reshape(-1, 1, 3), img_points.reshape(-1, 1, 2), K, None, reprojectionError=8, iterationsCount=100)#, flags=cv2.SOLVEPNP_EPNP)
#retval, rvec, tvec = cv2.solveP3P(pts.reshape(-1, 1, 3), img_points.reshape(-1, 1, 2), K, None, flags=cv2.SOLVEPNP_P3P)
#rvec = rvec[0]
#tvec = tvec[0]
print(retval)

# convert t vector from camera coords to world coords
R_recovered = cv2.Rodrigues(rvec)[0].T
t_recovered = -np.matmul(cv2.Rodrigues(rvec)[0].T, tvec)

print(R_recovered, t_recovered)

# expected:
# rvec, tvec is the position and orientation of the world coords w.r.t. the coordinate system of the second camera
# to get pose of second camera w.r.t. world coords compute -Rodrigues(rvec).T*tvec

2
[[-0.04427829 -0.02911439  0.99859491]
 [ 0.65984986  0.74965689  0.05111465]
 [-0.75009173  0.66118597 -0.0139824 ]] [[-6.17432281]
 [ 0.09153759]
 [ 9.62433663]]


In [142]:
R2-R_recovered

array([[ 1.04427829,  0.02911439, -0.99859491],
       [-0.65984986,  0.25034311, -0.05111465],
       [ 0.75009173, -0.66118597,  1.0139824 ]])

In [143]:
t2-t_recovered

array([[ 7.17432281],
       [-0.09153759],
       [-9.62433663]])

In [116]:
# map a world point to a camera point for testing
for pt in pts:
    test_pts = cv2.convertPointsToHomogeneous(pt.reshape(1, 3))
    R_recovered = cv2.Rodrigues(rvec)[0]
    t_recovered = tvec
    proj_matric_recovered = np.zeros((3, 4))
    proj_matric_recovered[:, :3] = R_recovered
    proj_matric_recovered[:, -1] = t_recovered.reshape(3,)
    proj_matric_recovered = np.matmul(K, proj_matric_recovered)
    img_pts = np.matmul(proj_matric_recovered, test_pts[0, 0, :]) 
    img_pts = cv2.convertPointsFromHomogeneous(img_pts.reshape(1, 3))
    print("world", pt, "reprojected image", img_pts)

world [-5.e-323 -0.e+000  1.e+001] reprojected image [[[0.40000002 0.49999999]]]
world [ 1.  1. 10.] reprojected image [[[0.50000002 0.59999999]]]
world [1.79588498 0.42833784 8.38445426] reprojected image [[[0.59999998 0.40000002]]]
world [ 2.94885129  3.91628023 19.53544818] reprojected image [[[0.70099566 1.22408603]]]


In [117]:
rvec2, _ = cv2.Rodrigues(R2.T)
p2, _ = cv2.projectPoints(pts, rvec2, -t2, K, distCoeffs=None)
print(p2)

[[[0.4        0.5       ]]

 [[0.5        0.6       ]]

 [[0.59492389 0.55108715]]

 [[0.59975974 0.70047046]]]


In [118]:
p2, _ = cv2.projectPoints(pts, rvec, tvec, K, distCoeffs=None)
print(p2)

[[[0.40000002 0.49999999]]

 [[0.50000002 0.59999999]]

 [[0.59999998 0.40000002]]

 [[0.70099566 1.22408603]]]


In [119]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2])
plot_basis(ax, R1, t1.reshape(3,))
plot_basis(ax, R2, t2.reshape(3,))
plot_basis(ax, R_recovered, t_recovered.reshape(3,))
ax.set_xlim([-10,10])
ax.set_ylim([-10,10])
ax.set_zlim([-10,10])
plt.show()

FigureCanvasNbAgg()